# 조회에 필요한 데이터 불러오기

In [5]:
from tqdm import tqdm
import pandas as pd
from glob import glob

In [3]:
files = sorted(glob('./data/*.csv'))
date_cols = ['date']
data_all = pd.DataFrame()
for file in tqdm(files):
    df_day = pd.read_csv(file, parse_dates=date_cols, usecols=['press', 'title', 'date', 'link']) # date 컬럼 datetime 형식으로 불러오기
    data_all = pd.concat([data_all, df_day])
data_all.reset_index(drop=True, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████| 730/730 [02:08<00:00,  5.68it/s]


In [4]:
data_all

,press,title,date,link
0,중앙일보,프로배구 2·3일 경기 연기.. 중계방송 스태프 코로나19 확진,2021-01-01,https://v.daum.net/v/20210101233343622
1,세계일보,"""산에서 커피 팔고 싶어"" 조두순의 소박한 꿈 이뤄질까? 시민들 '불안'",2021-01-01,https://v.daum.net/v/20210101233139613
2,서울신문,확진 1000명 목전 두고..秋 '구치소 집단감염' 첫 사과(종합),2021-01-01,https://v.daum.net/v/20210101233101608
3,국민일보,문건 유출? 미루기?.. 벌써 다 퍼진 '2.5단계 연장안',2021-01-01,https://v.daum.net/v/20210101231448533
4,뉴스1,'해변 폐쇄·드론 감시' 강릉 해맞이객 90% 줄어..그래도 1만6천명,2021-01-01,https://v.daum.net/v/20210101231238518
...,...,...,...,...
1411408,세계일보,"한국남동발전, 8년째 취약계층 평생학습 지원",2022-12-31,https://v.daum.net/v/20221231010122552
1411409,세계일보,"KIS제주, 하버드대 2년 연속 합격생 배출",2022-12-31,https://v.daum.net/v/20221231010117551
1411410,한국일보,늘어나는 방음터널... 번지면 대형 화재인데 방염 규정은 전무,2022-12-31,https://v.daum.net/v/20221231001003411
1411411,국민일보,[핫인픽] #배달할증료 #한파·폭설 출근길 #메시 지폐,2022-12-31,https://v.daum.net/v/20221231000423387


# 전체 데이터 유사도 추천

## 모델 불러오기

In [9]:
from sentence_transformers import SentenceTransformer
import torch
import pickle

In [7]:
model = SentenceTransformer('ddobokki/klue-roberta-small-nli-sts')

In [10]:
with open("./model/pickled_embeddings.bin", "rb") as f:
    embeddings = torch.load(f, map_location=torch.device('cpu'))

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

## 형태소 추출

In [28]:
def string_tokenizer(content):
    import pandas as pd
    from konlpy.tag import Okt
    okt = Okt()
    
    content = pd.Series(content)
    # 한글, 숫자, 공백만 남기기
    content = content.str.replace('[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', regex=True)
    token_content = okt.pos(content[0], norm=True, stem=True)
    return token_content

In [29]:
except_list = ['Exclamation', 'Josa', 'KoreanParticle', 'Determiner',  'Eomi', 'Suffix',  'VerbPrefix', 'PreEomi']
include_list = ['Verb',  'Noun']

In [30]:
# 230207 기현
def tag_except_list(except_list, token_content):
    filtered_list= []
    for tag in token_content :
        if tag[1] in except_list:
            continue
        filtered_list.append(tag[0])
    content = " ".join(filtered_list)
    return content

In [31]:
# 230207 기현
def tag_include_list(include_list, token_content):
    filtered_list=[]
    for tag in token_content :
        if tag[1] in include_list:
            filtered_list.append(tag[0])

    content = " ".join(filtered_list)
    return content

## 다음 url로 유사 기사 추천

In [32]:
stop_content = ["무단전재", "재배포금지","저작권자 ⓒ 서울신문사","무단복제 및 전재","무단 전재 및 재배포","제보는 카톡", "☞", "무단 전재-재배포", "▶연합뉴스 앱 지금 바로 다운받기~"]
rep_list = ['기사내용 요약']

In [33]:
def create_soup(url):
    from bs4 import BeautifulSoup
    import requests
    i = 0
    ## 요청 오류시 10번  재시도
    while i < 10 :
        try:
            res = requests.get(url)
            res.raise_for_status()
            soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
            break
        except:
            i += 1
            
    return soup

In [34]:
def content_only_scraper(link):
    article_soup = create_soup(link)

    # 본문 정리
    article = article_soup.find_all('section')[1]
    content = article.find_all(True, attrs={'dmcf-ptype':'general'})
    rst = []
    for para in content:
        for tmp in para.text.split('\n'):
            if tmp.strip() != '':
                rst.append(tmp.strip())
    content = []
    for c in rst:
        for i in stop_content:
            if i in c:
                break
        else:
            for rep in rep_list:
                c = c.replace(rep, '')
            content.append(c)
    content = ' '.join(content)
    if len(content) < 200:
        return False
    
    return content

In [35]:
def get_recommend_by_url(url, n, drange=False):
    from sentence_transformers import util
    # 다음 뉴스인지 확인
    if not url.startswith('https://v.daum.net/v/'):
        return False
    # 본문 스크랩
    content = content_only_scraper(url)

    # 형태소 전처리
    token = tag_except_list(except_list, string_tokenizer(content))
    
    embeded_content = model.encode([token], convert_to_tensor=True)
    
    # 기간 설정
    if drange:
        data_filtered_idx = data_all[data_all['date'].isin(pd.date_range(drange[0], drange[1]))].index
        embeddings_filtered = embeddings[data_filtered_idx]
        cos_sim_res = util.cos_sim(embeded_content, embeddings_filtered)
    else:
        cos_sim_res = util.cos_sim(embeded_content, embeddings)
    
    # 유사도 정렬, 추출
    sim_scores = list(enumerate(cos_sim_res[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_n = sim_scores[1:n+1]
    # sim_scores_n = [sim for sim in sim_scores_n if sim[1] >= 0.6] # 유사도 60% 이상 필터=>옵션화 가능?###############
    
    # 해당하는 기사 가져오기
    article_idx = [article_dict[0] for article_dict in sim_scores_n]
    rst = data_all.iloc[article_idx][['press', 'title', 'date', 'link']]
    rst['similarity'] = [round(sim[1].item(), 4)*100 for sim in sim_scores_n]
    
    return rst[['press', 'similarity', 'title', 'date', 'link']]

In [36]:
%%time
url = """https://v.daum.net/v/20230206162031399"""
get_recommend_by_url(url, 10)

CPU times: total: 18 s
Wall time: 21.8 s


,press,similarity,title,date,link
339634,문화일보,17.53,'피고인' 조국·정경심 나란히 법정 출석,2021-06-11,https://v.daum.net/v/20210611112009228
1346082,뉴시스,17.36,"조국 입시비리 등 의혹, 이번주 결심…이르면 올해 1심 결론",2022-11-27,https://v.daum.net/v/20221127095726043
252388,뉴시스,16.79,"조국·정경심, 6월11일 법정 같이선다..반년만 재판 재개",2021-04-30,https://v.daum.net/v/20210430121651371
252178,국민일보,16.42,조국 재판 6개월 만에 재개..정경심과 법정 같이 설듯,2021-04-30,https://v.daum.net/v/20210430140850136
327314,뉴시스,16.06,"조국·정경심, 나란히 법정 선다..6개월만에 재판 재개",2021-06-06,https://v.daum.net/v/20210606050052336
339727,조선일보,15.33,"'길고도 험한 길'.. 조국, 법정 출석 前 비틀스 노래 공유",2021-06-11,https://v.daum.net/v/20210611105704063
1357622,뉴시스,15.25,"검찰, 조국 '자녀 입시비리·감찰무마 혐의' 징역 5년 구형",2022-12-02,https://v.daum.net/v/20221202143703564
327183,국민일보,14.97,조국·정경심 나란히 법정 선다..6개월 만에 재판 재개,2021-06-06,https://v.daum.net/v/20210606085500027
1357681,뉴시스,14.52,"'자녀 입시비리' 조국 전 장관, 1심 결심공판 출석 [뉴시스Pic]",2022-12-02,https://v.daum.net/v/20221202141500738
339614,뉴시스,14.42,"조국 측 ""검찰의 투망식 공소""..'감찰무마' 거듭 부인",2021-06-11,https://v.daum.net/v/20210611112445421


## 직접 입력한 본문(문자열)로 유사 기사 추천

In [37]:
def get_recommend_by_content(content, n, drange=False):
    from sentence_transformers import util
    # 형태소 전처리
    token = tag_except_list(except_list, string_tokenizer(content))
    
    embeded_content = model.encode([token], convert_to_tensor=True)
    
    # 기간 설정
    if drange:
        data_filtered_idx = data_all[data_all['date'].isin(pd.date_range(drange[0], drange[1]))].index
        embeddings_filtered = embeddings[data_filtered_idx]
        cos_sim_res = util.cos_sim(embeded_content, embeddings_filtered)
    else:
        cos_sim_res = util.cos_sim(embeded_content, embeddings)
    
    # 유사도 정렬, 추출
    sim_scores = list(enumerate(cos_sim_res[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_n = sim_scores[1:n+1]
    # sim_scores_n = [sim for sim in sim_scores_n if sim[1] >= 0.6] # 유사도 60% 이상 필터=>옵션화 가능?###############
    
    # 해당하는 기사 가져오기
    article_idx = [article_dict[0] for article_dict in sim_scores_n]
    rst = data_all.iloc[article_idx][['press', 'title', 'date', 'link']]
    rst['similarity'] = [round(sim[1].item(), 4)*100 for sim in sim_scores_n]
        
    return rst[['press', 'similarity', 'title', 'date', 'link']]

In [39]:
%%time
arc_content = """2011년 한국을 떠나 러시아로 귀화한 전 쇼트트랙 선수 빅토르 안(38·한국명 안현수)의 국내 복귀 시도가 일단 무산됐다. 경기도 성남시청 직장운동부 빙상팀 코치에 지원했으나, 최종 후보에 들지 못해서다.

성남시는 29일 “시청 빙상팀 코치직 채용 전형에 빅토르 안을 포함해 7명이 지원했는데, 빅토르 안은 상위 2배수 후보에 들지 않았다”고 밝혔다. 시 관계자는 “서류와 면접 심사를 통해 기술, 소통 능력 등 여러 요소를 종합해 판단했다”며 “빙상계 여론과 언론 보도 등을 통해 나오는 시각도 평가에 반영됐다”고 했다.

성남시는 지난해 12월 19일 빙상팀 코치를 뽑기 위한 채용 공고를 냈다. 빅토르 안과 베이징 동계올림픽에서 중국 대표팀을 이끈 김선태 전 감독 등 7명이 지원했다. 김 전 감독도 최종 후보에 들지 못한 것으로 전해졌다.

빅토르 안은 2006년 토리노 동계올림픽에서 3개의 금메달을 딴 한국 쇼트트랙의 간판이었다. 2011년 소속팀이었던 성남시청이 빙상팀을 해체하자 선수 생활을 이어가기 위해 러시아로 귀화했다. 2014년 소치 동계올림픽에는 러시아 선수로 나서 3관왕에 올랐다. 2018년 평창 동계올림픽 출전이 무산된 이후에는 은퇴를 선언하고 지도자로 변신했다. 2022년 베이징 동계올림픽에서는 중국 대표팀 코치로 활동했다.

성남시는 31일 빙상팀 코치 선발 결과를 발표할 예정이다."""
get_recommend_by_content(arc_content, 10)

CPU times: total: 8.09 s
Wall time: 8.23 s


,press,similarity,title,date,link
814217,동아일보,34.96,"원희룡 ""안현수 귀화, 이재명 탓""..李측 ""거짓말 매일 진화""",2022-02-09,https://v.daum.net/v/20220209143615361
150711,조선일보,34.51,"연금 챙긴 빅토르 안, 빈손 임효준.. 같은 귀화인데, 무슨 차이?",2021-03-14,https://v.daum.net/v/20210314104031586
812691,뉴스1,30.88,"""선배 대우 원하냐""..안현수, 韓대표팀 쓰담쓰담 격려에 '뭇매'[영상]",2022-02-08,https://v.daum.net/v/20220208100049059
1326853,한국일보,29.23,빙상 스포츠 불모지 남양주에 ‘빙상경기연맹’ 출범,2022-11-17,https://v.daum.net/v/20221117205254511
814859,세계일보,27.44,"'편파판정' 논란, 삼성 때문이라는 안민석.. 체육계 ""왜 손 뗐는지 자문해보길""",2022-02-09,https://v.daum.net/v/20220209112610427
558522,세계일보,27.27,"대한빙상경기연맹 윤홍근 회장, 빙상종목 국가대표 지도자 및 선수단 격려",2021-09-29,https://v.daum.net/v/20210929015144774
815571,세계일보,26.63,쇼트트랙 대표팀 전임 코치 체제로 출전.. 감독 선발 기준 얼마나 높길래 [이슈+],2022-02-09,https://v.daum.net/v/20220209060543941
1049729,한국일보,26.55,"최민정·김민석·유영, 2022년 빙상연맹 최우수선수상",2022-06-17,https://v.daum.net/v/20220617082830662
722004,한국일보,26.29,"""전명규 강압훈련에 반발 조재범, 코치 되더니 폭력 대물림"" [일그러진 스포츠]",2021-12-21,https://v.daum.net/v/20211221110003388
1337775,한겨레,26.18,빅토르 최 탄생 60돌 24일 토크콘서트,2022-11-23,https://v.daum.net/v/20221123194504622


In [38]:
%%time
arc_content = """19년 간 2억원이 넘는 기부를 실천한 사람이 있다.

지난 2004년부터 매년 꾸준히 어려운 이웃을 위한 나눔을 실천, 19년간 2억 2000만원을 기부한 시민이 있어 지역사회의 귀감이 되고 있다.

코로나19가 길어져 취약 계층의 복지 수요가 늘었다고 판단한 황 대표는 올해 1월부터는 기부액을 매달 200만원으로 늘렸다.

이렇게 이어온 황 대표의 누적 기부 금액은 2억 2000만원에 달한다."""
get_recommend_by_content(arc_content, 10)

CPU times: total: 9.47 s
Wall time: 9.57 s


,press,similarity,title,date,link
1125112,뉴스1,25.02,19년 동안 2억2000만원 내 놓은 기부천사 황영희씨,2022-07-28,https://v.daum.net/v/20220728101841849
710321,중앙일보,12.73,[속보] 윤갑근 전 고검장 '라임 로비' 혐의 2심 무죄,2021-12-15,https://v.daum.net/v/20211215113100728
897616,뉴스1,10.45,강남아파트서 억대 절도 40대 남성 검찰 송치,2022-03-25,https://v.daum.net/v/20220325081710727
897621,뉴스1,10.45,압구정 아파트 돌며 2억대 현금·금품 훔친 40대 남성 검거,2022-03-25,https://v.daum.net/v/20220325081524702
897622,뉴스1,10.45,강남아파트서 '2억2000만원' 절도 40대 남성 검거,2022-03-25,https://v.daum.net/v/20220325081522701
897623,뉴스1,10.45,강남아파트 돌며 2억대 현금·금품 훔친 40대 남성 검거,2022-03-25,https://v.daum.net/v/20220325081520700
897624,뉴스1,10.45,강남아파트서 '2억2000만원' 절도 40대 남성 검찰 송치,2022-03-25,https://v.daum.net/v/20220325081511696
897625,뉴스1,10.45,강남아파트서 '2억2000만원' 절도 40대 남성 검거,2022-03-25,https://v.daum.net/v/20220325081458692
897626,뉴스1,10.45,강남아파트 돌며 '2억2000만원' 절도 40대 남성 검찰 송치,2022-03-25,https://v.daum.net/v/20220325081453690
271380,뉴스1,10.36,'라임 로비' 윤갑근 前고검장 징역 3년 불복 항소,2021-05-10,https://v.daum.net/v/20210510082829198
